# 1. Importación de paquetes

In [105]:
#Import packages
from modules import preprocess
from modules import training

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [106]:
df = preprocess.load_csv_data('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [107]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [108]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [109]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int32

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [110]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *train* y *test*

In [111]:
# Split dataset into train and test
train_df, test_df = preprocess.split_dataset(df)
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [112]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
8054,25.13,25.13,24.69,25.31,6,6,32,21,0.08,0.05,0.06,0.09,350,0.000000,0,0,0
6125,25.44,25.44,25.38,26.13,5,6,30,21,0.08,0.05,0.06,0.09,350,0.107692,0,0,0
1739,25.31,25.31,24.81,25.63,0,0,0,0,0.07,0.06,0.06,0.06,365,-0.050000,0,0,0
2899,25.63,25.31,25.06,25.94,124,38,72,52,0.54,0.18,0.18,0.13,370,0.153846,0,0,1
9382,25.19,25.19,24.69,25.25,0,0,0,0,0.08,0.04,0.06,0.08,345,0.000000,0,0,0


In [113]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
3056,26.00,26.50,25.44,26.31,162,249,84,62,0.88,0.47,0.39,0.25,565,4.323077,1,1,1
6146,25.44,25.44,25.38,26.19,5,5,24,17,0.08,0.07,0.06,0.10,355,0.080769,0,0,0
1385,25.44,25.50,25.00,25.75,0,0,0,0,0.06,0.05,0.06,0.06,385,-0.280769,0,0,0
171,25.69,28.13,25.13,26.06,157,244,80,61,0.21,0.27,0.13,0.14,635,1.557692,1,0,1
9915,25.06,25.06,24.56,25.13,0,0,4,2,0.08,0.05,0.06,0.09,345,0.000000,0,0,0


## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [114]:
# Get features columns
X = preprocess.get_features(train_df)

# Get output column
y = preprocess.get_output(train_df)

In [115]:
X.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
8054,25.13,25.13,24.69,25.31,6,6,32,21,0.08,0.05,0.06,0.09,350,0.000000,0,0
6125,25.44,25.44,25.38,26.13,5,6,30,21,0.08,0.05,0.06,0.09,350,0.107692,0,0
1739,25.31,25.31,24.81,25.63,0,0,0,0,0.07,0.06,0.06,0.06,365,-0.050000,0,0
2899,25.63,25.31,25.06,25.94,124,38,72,52,0.54,0.18,0.18,0.13,370,0.153846,0,0
9382,25.19,25.19,24.69,25.25,0,0,0,0,0.08,0.04,0.06,0.08,345,0.000000,0,0


In [116]:
y.head()

8054    0
6125    0
1739    0
2899    1
9382    0
Name: Room_Occupancy_Count, dtype: int32

# 2. Entrenamiento y seguimiento de la huella de carbono con **CodeCarbon**

In [117]:
# Logistic Regression
training.train_LR_codecarbon(X,y)

[codecarbon INFO @ 11:03:22] [setup] RAM Tracking...
[codecarbon INFO @ 11:03:22] [setup] GPU Tracking...
[codecarbon INFO @ 11:03:22] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:03:22] [setup] CPU Tracking...
[codecarbon WARNING @ 11:03:22] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:03:24] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:03:24] >>> Tracker's metadata:
[codecarbon INFO @ 11:03:24]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 11:03:24]   Python version: 3.10.4
[codecarbon INFO @ 11:03:24]   Available RAM : 7.935 GB
[codecarbon INFO @ 11:03:24]   CPU count: 6
[codecarbon INFO @ 11:03:24]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:03:24]   GPU count: 1
[codecarbon INFO @ 11:03:24]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 11:03:24] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W


Emissions: 1.1391997165359493e-07 kg


In [118]:
# Random Forest
training.train_RF_codecarbon(X,y)

[codecarbon INFO @ 11:03:24] [setup] RAM Tracking...
[codecarbon INFO @ 11:03:24] [setup] GPU Tracking...
[codecarbon INFO @ 11:03:24] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:03:24] [setup] CPU Tracking...
[codecarbon WARNING @ 11:03:24] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:03:26] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:03:26] >>> Tracker's metadata:
[codecarbon INFO @ 11:03:26]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 11:03:26]   Python version: 3.10.4
[codecarbon INFO @ 11:03:26]   Available RAM : 7.935 GB
[codecarbon INFO @ 11:03:26]   CPU count: 6
[codecarbon INFO @ 11:03:26]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:03:26]   GPU count: 1
[codecarbon INFO @ 11:03:26]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 11:03:27] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W


Emissions: 3.5979159519385147e-07 kg


In [119]:
# Linear SVC (Support Vector Machines)
training.train_SVC_codecarbon(X,y)

[codecarbon INFO @ 11:03:27] [setup] RAM Tracking...
[codecarbon INFO @ 11:03:27] [setup] GPU Tracking...
[codecarbon INFO @ 11:03:27] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:03:27] [setup] CPU Tracking...
[codecarbon WARNING @ 11:03:27] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:03:29] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:03:29] >>> Tracker's metadata:
[codecarbon INFO @ 11:03:29]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 11:03:29]   Python version: 3.10.4
[codecarbon INFO @ 11:03:29]   Available RAM : 7.935 GB
[codecarbon INFO @ 11:03:29]   CPU count: 6
[codecarbon INFO @ 11:03:29]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:03:29]   GPU count: 1
[codecarbon INFO @ 11:03:29]   GPU model: 1 x NVIDIA GeForce GT 710
C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\sklearn\svm\_base.py:1225: Convergen

Emissions: 3.962732818174668e-08 kg
